In [184]:
!pip3 install --user google-cloud-automl==0.9.0

  Using cached https://files.pythonhosted.org/packages/5f/60/d2e5721713967d40cdee5ed9da4fad4a0349dada84608f16f24920afc6f2/google_cloud_automl-0.9.0-py2.py3-none-any.whl
  Found existing installation: google-cloud-automl 0.4.0
    Uninstalling google-cloud-automl-0.4.0:
      Successfully uninstalled google-cloud-automl-0.4.0


In [1]:
#! pip3 install --user kfp --upgrade
#! pip3 install --user google-cloud-automl

In [40]:
import kfp
client = kfp.Client(host='4709991555f6b0ae-dot-us-central1.notebooks.googleusercontent.com')

# AutoML Serverless ML (Taxi) on KFP

TODO: set GCS location to experiment run
TODO: set num passengers out of categorical
TODO: run on unsampled datset
TODO: set split column to use fingerprint
TODO: Difference between table and dataset?
TODO: try using a cleanup task with boolean in testing mode

In [4]:
# Configuration

PROJECT_ID = 'dhodun1'
COMPUTE_REGION = 'us-central1' # Currently us-central1 is only region
BUCKET = 'dhodun1-central1'

# Raw dataset, not cleaned
QUERY = '''
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers
FROM `nyc-tlc.yellow.trips`
WHERE MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 100000) = 1
'''


In [41]:
# AutoML Tables components

import kfp.components.gcp.automl.


In [20]:
component_store = comp.ComponentStore(local_search_paths=['./pipelines/components'])
component_store.load_component('gcp/automl/create_dataset_for_tables')

<function Automl create dataset for tables(gcp_project_id:str, gcp_region:str, display_name:str, description:str=None, tables_dataset_metadata:dict='{}', retry=None, timeout:float=None, metadata:dict=None)>

In [6]:
component_store = comp.ComponentStore(local_search_paths=['./pipelines/components'])

In [7]:
automl_create_dataset_for_tables_op = component_store.load_component('gcp/automl/create_dataset_for_tables')
automl_import_data_from_bigquery_op = component_store.load_component('gcp/automl/import_data_from_bigquery')
automl_import_data_from_gcs_op = component_store.load_component('gcp/automl/import_data_from_gcs')
automl_create_model_for_tables_op = component_store.load_component('gcp/automl/create_model_for_tables')
prediction_service_batch_predict_op = component_store.load_component('gcp/automl/prediction_service_batch_predict')
automl_split_dataset_table_column_names_op = component_store.load_component('gcp/automl/split_dataset_table_column_names')

bigquery_query_op = component_store.load_component('gcp/bigquery/query')
#automl_create_dataset_for_tables_op = comp.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/b3179d86b239a08bf4884b50dbf3a9151da96d66/components/gcp/automl/create_dataset_for_tables/component.yaml')

In [8]:
#automl_create_dataset_for_tables_op(PROJECT_ID, COMPUTE_REGION, 'taxi_data',)

## Create Op to change 'passengers' from categorical to numeric
Should be fixed with *1.0 in BQ Query, but related to this BQ CSV export bug: https://b.corp.google.com/issues/143356550

In [30]:
dataset_path = 'projects/978546835329/locations/us-central1/datasets/TBL5712864505831096320'

In [38]:
from typing import NamedTuple

def set_automl_tables_column_type(
    dataset_path: str,
    column_display_name: str,
    type_code: str,    
):
    # Updates AutuML Column with new column type, does trigger a new column statistics job? how do we check?
    import sys
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'google-cloud-automl==0.9.0', '--quiet', '--no-warn-script-location'], env={'PIP_DISABLE_PIP_VERSION_CHECK': '1'}, check=True)
    
    from google.cloud import automl_v1beta1
    client = automl_v1beta1.TablesClient()
    
    dataset = client.get_dataset(dataset_name=dataset_path)
    
    column_specs_response = client.list_column_specs(dataset=dataset)
    column_specs = list(column_specs_response)
    
    for column in column_specs:
        if column.display_name == column_display_name:
            # This kicks off a new statistics job... how to check to see if it's done? Took ~ 1 minute this time
            response = client.update_column_spec(column_spec_name=column.name, dataset=dataset, type_code=type_code)
            result = response.result
            print(result)
            metadata = response.metadata
            print(metadata)
            #print('Updated column: {} to type code {}. Generating new statistics now...').format()

    
    

In [39]:
set_automl_tables_column_type(dataset_path=dataset_path, column_display_name='passengers', type_code='FLOAT64')

AttributeError: result

In [9]:
# Define the pipeline

import kfp
from kfp import dsl

def serverless_ml_taxi_pipeline(
    gcp_project_id: str,
    gcp_region: str,
    query: str,
    gcs_bucket: str,
    gcs_temp_directory: str = 'ml-taxi/{}'.format(dsl.RUN_ID_PLACEHOLDER),
    dataset_display_name: str = 'taxi_data',
    dataset_location: str = 'US',
    target_column_name: str = 'fare_amount',
    model_display_name: str = 'taxi_data_model',
    train_budget_milli_node_hours: 'Integer' = 1000,
):
    
    output_gcs_path='gs://{}/{}/bq_taxi_output.csv'.format(gcs_bucket, gcs_temp_directory)
    '''
    # Create dataset
    create_dataset_task = automl_create_dataset_for_tables_op(
        gcp_project_id=gcp_project_id,
        gcp_region=gcp_region,
        display_name=dataset_display_name,
    )
    
    # Query to clean dataset and dump to GCS
    bigquery_export_task = bigquery_query_op(
        query=query,
        project_id=gcp_project_id,
        output_gcs_path=output_gcs_path,
        dataset_location=dataset_location,
    )
    
    # Import data from GCS automl_import_data_from_gcs_op
    import_data_task = automl_import_data_from_gcs_op(
        dataset_path=create_dataset_task.outputs['dataset_path'],
        input_uris=[output_gcs_path],
    ).after(bigquery_export_task)
    '''
    # Prepare column schemas
    split_column_specs_task = automl_split_dataset_table_column_names_op(
        #dataset_path=import_data_task.outputs['dataset_path'],
        dataset_path='projects/978546835329/locations/us-central1/datasets/TBL5712864505831096320',
        table_index=0,
        target_column_name=target_column_name,        
    )
    
    # Train a model
    create_model_task = automl_create_model_for_tables_op(
        gcp_project_id=gcp_project_id,
        gcp_region=gcp_region,
        display_name=model_display_name,
        #dataset_id=create_dataset_task.outputs['dataset_id'],
        dataset_id='TBL5712864505831096320',
        target_column_path=split_column_specs_task.outputs['target_column_path'],
        #input_feature_column_paths=None, # All non-target columns will be used if None is passed
        input_feature_column_paths=split_column_specs_task.outputs['feature_column_paths'],
        optimization_objective='MINIMIZE_RMSE',
        train_budget_milli_node_hours=train_budget_milli_node_hours,
    )
    
    from kfp.gcp import use_gcp_secret
    kfp.dsl.get_pipeline_conf().add_op_transformer(use_gcp_secret('user-gcp-sa'))

In [13]:
# Run the pipeline

from google.cloud import automl

#location_path = automl.AutoMlClient().location_path(PROJECT_ID, COMPUTE_REGION)

In [14]:
client = kfp.Client(host='4709991555f6b0ae-dot-us-central1.notebooks.googleusercontent.com')

In [15]:
from google.cloud import automl

#PROJECT_ID = 'dhodun1'
#COMPUTE_REGION = 'us-central1' # Currently us-central1 is only region

#location_path = automl.AutoMlClient().location_path(PROJECT_ID, COMPUTE_REGION)

client.create_run_from_pipeline_func(
    serverless_ml_taxi_pipeline,
    arguments=dict(
        gcp_project_id=PROJECT_ID,
        gcp_region=COMPUTE_REGION,
        query=QUERY,
        gcs_bucket=BUCKET,
        dataset_display_name='taxi_data',
    )
)

RunPipelineResult(run_id=ddd0f244-a015-47de-8785-6310e30a660a)

In [100]:
!gcloud container clusters list

NAME                    LOCATION       MASTER_VERSION  MASTER_IP      MACHINE_TYPE   NODE_VERSION     NUM_NODES  STATUS
kubeflow-marketplace-1  us-central1-a  1.13.11-gke.14  34.67.6.60     n1-standard-2  1.13.11-gke.14   6          RUNNING
kubeflow-test           us-central1-a  1.12.10-gke.17  34.67.124.179  n1-standard-8  1.11.10-gke.5 *  2          RUNNING
mykfp4                  us-central1-a  1.12.10-gke.17  34.66.115.116  n1-standard-8  1.11.10-gke.5 *  2          RUNNING
mykfp                   us-central1-b  1.12.10-gke.17  35.192.72.217  n1-standard-2  1.12.10-gke.17   7          RUNNING

* - There is an upgrade available for your cluster(s).

To upgrade nodes to the latest available version, run
  $ gcloud container clusters upgrade NAME


In [101]:
!gcloud container clusters get-credentials kubeflow-marketplace-1 --zone us-central1-a

Fetching cluster endpoint and auth data.
kubeconfig entry generated for kubeflow-marketplace-1.


In [2]:
dataset_path = 'projects/978546835329/locations/us-central1/datasets/TBL5712864505831096320'
client = automl.AutoMlClient()
list_table_specs_response = client.list_table_specs(dataset_path)
list_table_specs_response
table_specs = [s for s in list_table_specs_response]
print('table_specs=')
print(table_specs)
table_spec_name = table_specs[0].name

list_column_specs_response = client.list_column_specs(table_spec_name)
column_specs = [s for s in list_column_specs_response]
#client.get_column_spec('passengers')
for column in column_specs:
    if column.display_name == 'passengers':
        passenger_column = column.name

print('column_specs=')
#print(column_specs)

column = client.get_column_spec(passenger_column)
print(column)

#client = automl_v1beta1.TablesClient()

client.update_column_spec(column_spec_name=passenger_column, type_code='NUMERIC')

NameError: name 'automl' is not defined

In [171]:
column.name

'projects/978546835329/locations/us-central1/datasets/TBL5712864505831096320/tableSpecs/5624547333841616896/columnSpecs/6968091169355988992'

In [122]:
table_specs[]

SyntaxError: invalid syntax (<ipython-input-122-9b1fbe13ef43>, line 1)

In [102]:
kfp.run_pipeline_func_on_cluster(
    serverless_ml_taxi_pipeline,
    arguments = dict(
        gcp_project_id=PROJECT_ID,
        gcp_region=COMPUTE_REGION,
        display_name='taxi_data'
    )
)

ApiException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Date': 'Thu, 21 Nov 2019 16:48:58 GMT', 'Audit-Id': '33ad3025-ca11-499b-99af-aa913332563e', 'Content-Length': '198', 'Content-Type': 'application/json'})
HTTP response body: {"kind":"Status","apiVersion":"v1","metadata":{},"status":"Failure","message":"services \"ml-pipeline\" not found","reason":"NotFound","details":{"name":"ml-pipeline","kind":"services"},"code":404}



In [147]:
!pip install --user google-cloud-automl==0.4.0

  Using cached https://files.pythonhosted.org/packages/1a/d2/54d91e8bda501ea80126d1c46f255008bba4e03ce3fa4f3738511f96def1/google_cloud_automl-0.4.0-py2.py3-none-any.whl
  Found existing installation: google-cloud-automl 0.9.0
    Uninstalling google-cloud-automl-0.9.0:
      Successfully uninstalled google-cloud-automl-0.9.0


In [160]:
gcp_project_id='dhodun1'
gcp_region='us_central1'
display_name='taxi_data'


from typing import NamedTuple

def automl_create_dataset_for_tables(
    gcp_project_id: str,
    gcp_region: str,
    display_name: str,
    description: str = None,
    tables_dataset_metadata: dict = {},
    retry=None, #=google.api_core.gapic_v1.method.DEFAULT,
    timeout: float = None, #=google.api_core.gapic_v1.method.DEFAULT,
    metadata: dict = None,
) -> NamedTuple('Outputs', [('dataset_path', str), ('create_time', str), ('dataset_id', str)]):
    '''automl_create_dataset_for_tables creates an empty Dataset for AutoML tables
    '''
    import sys
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'google-cloud-automl==0.4.0', '--quiet', '--no-warn-script-location'], env={'PIP_DISABLE_PIP_VERSION_CHECK': '1'}, check=True)

    import google
    from google.cloud import automl
    client = automl.AutoMlClient()

    location_path = client.location_path(gcp_project_id, gcp_region)
    dataset_dict = {
        'display_name': display_name,
        'description': description,
        #'tables_dataset_metadata': tables_dataset_metadata,
    }
    dataset = client.create_dataset(
        location_path,
        dataset_dict,
        retry or google.api_core.gapic_v1.method.DEFAULT,
        timeout or google.api_core.gapic_v1.method.DEFAULT,
        metadata,
    )
    print(dataset)
    dataset_id = dataset.name.rsplit('/', 1)[-1]
    return (dataset.name, dataset.create_time, dataset_id)

In [161]:
automl_create_dataset_for_tables(gcp_project_id, gcp_region, display_name)

InvalidArgument: 400 List of found errors:	1.Field: parent; Message: The provided location ID is not valid.	2.Field: parent; Message: Required field is invalid	3.Field: dataset.dataset_metadata; Message: Required field not set.	

In [ ]:
client.create_dataset()